## Preamble
This is designed as a codified version of my notes on BI, and to demonstrate some python implementation of different noise models as likelihood functions. Useful references:

[1] [GW BI Introduction Paper](https://arxiv.org/pdf/1809.02293)

[2] [LIGO Noise Specs (some more good BI here)](https://arxiv.org/pdf/1908.11170)

In [7]:
import plotfancy as pf
import numpy as np
pf.housestyle_rcparams()

True

## General Gaussian Case ##

In the most general case, we relate our observed data $\textbf{X}$ from some model prediction $\boldsymbol{\mu}(\boldsymbol{\theta})$, based on model parameters $\boldsymbol{\theta}$ such that
\begin{equation}
\textbf{X} = \boldsymbol{\mu}(\boldsymbol{\theta}) + \boldsymbol{\mathscr{N}}
\end{equation}
for some noise vector $\boldsymbol{\mathscr{N}}$. The most general Gaussian case models $\boldsymbol{\mathscr{N}}$ as a multivariate gaussian - a vector whose elements are correlated, encoded with a covariance matrix, $\mathscr{C}$, relating the variance of each element
\begin{equation}
\mathscr{C} =
\begin{bmatrix}
\sigma^2(1) & \sigma(1,2) & ... & \sigma(1,K) \\
\sigma(2,1) & \sigma^2(2) & ... & \sigma(2,K) \\ 
... & ... & ... & ... \\
\sigma(K,1) & \sigma(K,2) & ... & \sigma^2(K)
\end{bmatrix}
\end{equation}
for $K = \texttt{dim}(\textbf{X})$ and $i,j\in\mathbb{Z}^+\leq K$ for $\mathscr{C}_{i,j}=\sigma(i,j)$ covariance. We establish the notation convention here that $\textbf{X}$, $\boldsymbol{\mu}(\boldsymbol{\theta})$, and $\boldsymbol{\mathscr{N}}$ are *frequency-domain* functions for the data, model and noise hereonwards unless explicitely noted otherwise as time-series.


Calculating the likelihood, which by definition follows $\mathbb{P}(\textbf{X}|\boldsymbol{\theta})$, here is equivalent to some $\mathbb{P}_{\boldsymbol{\mathscr{N}}}(\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta}))$ conditional probability, which follows from the first equation as our noise distribution;
\begin{equation}
\mathbb{P}(\textbf{X}|\boldsymbol{\theta}) =  \mathbb{P}_{\boldsymbol{\mathscr{N}}}(\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})) = \mathbb{P}_{\boldsymbol{\mathscr{N}}}(\boldsymbol{\mathscr{N}})
\end{equation}

Calculating this in a covariant case is non-trivial - we cannot simply combine probabilities through multiplication - instead we get a multivariate Gaussian of the form

\begin{equation}
\mathbb{P}_{MG} = \frac{1}{[(2\pi)^K\textnormal{det}(\mathscr{C})]^{1/2}} \exp \bigg[ -\frac{1}{2} [\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})]^T \mathscr{C}^{-1} [\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})] \bigg]
\end{equation}

We might model this most general case as a function:

In [ ]:
### Global params ###

fr = 100 # the number of frequency bins we'd have
n = 10 # number of example samples of noise

### FIRST FIND COVARIANCE - Observing no signal to create n frequency profiles of the noise ###

noise_samples = np.ones([n,fr]) # Dummy array
sample_mean = np.average(noise_samples, axis=0)
cen = noise_samples - sample_mean
cov = 1/(fr) * (np.transpose(cen) @ cen)

### We might then test this against some data... ###

data = np.ones([1,fr]) 
model = np.ones([1,fr]) # Dummy arrays we might observe from an experiment.

def lhood_MVG(X, MU, C):
    n = X-MU
    return 1/(2*np.pi*np.linalg.det(C))**(fr/2) * np.exp(-0.5*np.transpose(n)@ np.linalg.inv(C)@ n)

### Which would be implemented like ###

lhood_MVG(data, model, cov)

### We can, however, simplify in special cases:

The next simplest case is stationary noise; using $N(\mu,\sigma^2)$ to represent a normal distribution we express this case as noise following
\begin{equation}
\boldsymbol{\mathscr{N}}(t) \sim
\begin{bmatrix}
N(0,\sigma^2_1) \\ N(0,\sigma^2_2) \\ ... \\ N(0,\sigma^2_K)
\end{bmatrix}
\end{equation}

After the Fourier transform it follows that this then becomes some complex-valued noise - we make the assumption (easily verifiable by plotting the im-real correlation) that the imaginary and real parts are independently stochastic such that

\begin{equation}
\boldsymbol{\mathscr{N}} = \mathfrak{Re}(\boldsymbol{\mathscr{N}}) + i\mathfrak{Im}(\boldsymbol{\mathscr{N}}),
\end{equation}
more verbosely that
\begin{equation}
\boldsymbol{\mathscr{N}}(f) \sim
\begin{bmatrix}
N(0,\sigma^2_1) \\ N(0,\sigma^2_2) \\ ... \\ N(0,\sigma^2_K)
\end{bmatrix}
+ i
\begin{bmatrix}
N(0,\sigma^2_1) \\ N(0,\sigma^2_2) \\ ... \\ N(0,\sigma^2_K)
\end{bmatrix}
\end{equation}

in the frequency domain. Note that the real and imaginary elements must be thought as independently sampled from their respective distributions $N(0,\sigma^2_i)$: this is **not** $\mathscr{N}_i=N(0,\sigma^2_i)(1+i)$. This distribution has a diagonal covariance matrix and the frequency domain noise transforms to autocorrelated noise with some characteristic $\textbf{C}(t_i-t_j)$ correlation function in the time domain. We express $\sigma^2_i = S_{\boldsymbol{\mathscr{N}}}(f_i)$ where $S_{\boldsymbol{\mathscr{N}}}(f_i)$ is the power spectral density at some frequency bin $f_i$, estimated by taking $S_n(f_i) = \texttt{fft}(\boldsymbol{\mathscr{N}}(\tau))$ where $\tau$ is some characteristic $t_i\to t_j$ time period and $\boldsymbol{\mathscr{N}}(\tau)$ is the *time-series* noise. For this simplified covariance matrix 

\begin{equation}
\mathscr{C} =
\begin{bmatrix}
\sigma^2_1 & 0 & ... & 0 \\
0 & \sigma^2_2 & ... & 0 \\ 
... & ... & ... & ... \\ 
0 & 0 & ... & \sigma^2_K
\end{bmatrix}
\end{equation}

the determinant of this matrix then follows $\textnormal{det}(\mathscr{C}) = \prod^K_i \sigma_i$; because of the diagonality of the matrix we can further simplify the likelihood. By our assumption of complex-real stochastic independence, it follows that

\begin{aligned}
\textbf{X} &= \boldsymbol{\mu}(\boldsymbol{\theta}) + \mathfrak{Re}(\boldsymbol{\mathscr{N}}) + i\mathfrak{Im}(\boldsymbol{\mathscr{N}}) \\
\Rightarrow \mathbb{P}(\textbf{X}|\boldsymbol{\theta}) &=  \mathbb{P}_{\boldsymbol{\mathscr{N}}}(\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})) \\
&= \mathbb{P}_{\mathfrak{Re}(\boldsymbol{\mathscr{N}})}(\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})) \cdot \mathbb{P}_{\mathfrak{Im}(\boldsymbol{\mathscr{N}})}(\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})) \\
&= \prod^K_i \frac{1}{2\pi\sigma^2_i} \exp \bigg[ \frac{\mathfrak{Re}(\boldsymbol{\mathscr{N}}_i)^2}{2\pi\sigma^2_i} \bigg] \cdot \exp \bigg[ \frac{\mathfrak{Im}(\boldsymbol{\mathscr{N}}_i)^2}{2\pi\sigma^2_i} \bigg]  \\
&= \prod^K_i \frac{1}{2\pi\sigma^2_i} \exp \bigg[ \frac{\mathfrak{Re}(\boldsymbol{\mathscr{N}}_i)^2 + \mathfrak{Im}(\boldsymbol{\mathscr{N}}_i)^2}{2\pi\sigma^2_i} \bigg]
\end{aligned}

Noticing the encoded modulus-squared definition in the exponent argument
\begin{equation}
\mathfrak{Re}(\boldsymbol{\mathscr{N}}_i)^2 + \mathfrak{Im}(\boldsymbol{\mathscr{N}}_i)^2 \equiv |X - \mu(\boldsymbol{\theta})|^2_i
\end{equation}

we get to a simplified likelihood of
\begin{equation}
\mathbb{P}_{\textnormal{stat}} = \frac{1}{[\prod^K_i 2\pi\sigma_i]^{1/2}} \exp \bigg[ -\frac{1}{2} \sum^K_i \frac{[X-\mu(\boldsymbol{\theta})]^2_i}{\sigma^2_i} \bigg]
\end{equation}

Given this is now separable we can turn this into a *log likelihood* that follows
\begin{equation}
\log[\mathbb{P}_{\textnormal{stat}}(\textbf{X}|\boldsymbol{\theta})] = \sum^K_i\bigg[\frac{1}{2\pi\sigma_i} [X-\mu(\boldsymbol{\theta})]^2_i - \frac{1}{2} \log[{2\pi\sigma_i^2}]\bigg]
\end{equation}

Note that in literature the $\frac{1}{2} \log[{2\pi\sigma_i^2}]$ term is often omitted as it cancels under addition of likelihoods to create a (log) Bayes factor. With the exception of *glitches*, (short duration transients) and *adiabatic drift* (minute-to-hour drifts in the power spectrum), the noise at LIGO can be approximated as stationary [2]. We might implement this case as:

In [ ]:
### Global params ###

fr = 100 # the number of frequency bins we'd have
n = 10 # number of example samples of noise

### FIRST FIND COVARIANCE - This is simpler as we can just take the variance of each element ###

noise_samples = np.ones([n,fr]) # dummy array
cov = np.var(noise_samples, axis = 0)*np.identity(fr)

### We might then test this against some data... ###

data = np.ones([1,fr]) 
model = np.ones([1,fr]) # dummy arrays we might observe from an experiment.

def lhood_stat(X, MU, C):
    sigmas = np.diag(C)
    n = X-MU
    return 1/(np.prod((2*np.pi*sigmas)))**(0.5) * np.exp(-0.5*np.sum((n/sigmas)**2))
    
def log_lhood_stat(X,MU,C, ret_const=True): # we may also want to define the log-likelihood...
    sigmas = np.diag(C)
    const = (-0.5*np.sum(np.log((2*np.pi*sigmas**2)))) if ret_const else 0
    n = X-MU
    return np.sum(n**2/(2*np.pi*sigmas**2)) + const
    
### Which would be implemented like ###

lhood_stat(data, model, cov) #or
log_lhood_stat(data, model, cov)

### White Noise

The simplest case is white noise: where every element of the noise distribution follows the same normal distribution such that we can express

\begin{equation}
\boldsymbol{\mathscr{N}} = N(0,\sigma^2)
\begin{bmatrix}
1 \\ ... \\ 1
\end{bmatrix}
\end{equation}

and the covariance as 
\begin{equation}
\mathscr{C} = \sigma^2 \textbf{I} = S_{\boldsymbol{\mathscr{N}}} \textbf{I}
\end{equation}

Which causes simplfication of the above likelihoods to the following:

\begin{equation}
\mathbb{P}_{\textnormal{white}} = \frac{1}{[2\pi\sigma]^{K/2}} \exp \bigg[ -\frac{1}{2\sigma^2} \sum^K_i [X-\mu(\boldsymbol{\theta})]^2_i\bigg]
\end{equation}
\begin{equation}
\log[\mathbb{P}_{\textnormal{white}}(\textbf{X}|\boldsymbol{\theta})] = \frac{1}{2\pi\sigma} \sum^K_i\bigg[ [X-\mu(\boldsymbol{\theta})]^2_i \bigg] - \frac{K}{2} \log[{2\pi\sigma^2}]
\end{equation}

We should note that this is **not** a suitable model of the noise at LIGO [2]. Implementing it nevertheless follows:

In [ ]:
### Global params ###

fr = 100 # the number of frequency bins we'd have
n = 10 # number of example samples of noise

### FIRST FIND COVARIANCE - This is simpler as we can just take the variance of each element ###

noise_samples = np.ones([n,fr]) # dummy array
cov = np.var(noise_samples.flatten())*np.identity(fr)

### We might then test this against some data... ###

data = np.ones([1,fr]) 
model = np.ones([1,fr]) # dummy arrays we might observe from an experiment.

def lhood_white(X, MU, C):
    K = len(X)
    sigma = C[0,0]
    n = X-MU
    return 1/(2*np.pi*sigma)**(K*0.5) * np.exp(-0.5*np.sum((n/sigma)**2))
    
def log_lhood_white(X,MU,C, ret_const=True): # we may also want to define the log-likelihood...
    K = len(X)
    sigma = C[0,0]
    const = (-0.5*K*np.log((2*np.pi*sigma**2))) if ret_const else 0
    n = X-MU
    return np.sum(n**2/(2*np.pi*sigma**2)) + const
    
### Which would be implemented like ###

lhood_white(data, model, cov) #or
log_lhood_white(data, model, cov)

### What Happens When...?

Let's consider some modifications. Firstly when we apply a scaling vector to the noise, $\boldsymbol{\alpha}$. We then get
\begin{equation}
\textbf{X} = \boldsymbol{\mu}(\boldsymbol{\theta}) + \boldsymbol{\alpha} \boldsymbol{\mathscr{N}}
\end{equation}
Such that relation transforms to 
\begin{equation}
\boldsymbol{\alpha}^{-1}[\textbf{X} - \boldsymbol{\mu}(\boldsymbol{\theta})] = \boldsymbol{\mathscr{N}}
\end{equation}
In the static noise example this therefore us with an additional standard deviation on each element of the noise, so it follows that $\boldsymbol{\alpha}$ scales the power spectral density on each bin:
\begin{equation}
\mathbb{P}_{\textnormal{stat}} = \frac{1}{[\prod^K_i 2\pi\sigma_i]^{1/2}} \exp \bigg[ -\frac{1}{2} \sum^K_i \frac{[X-\mu(\boldsymbol{\theta})]^2_i}{\sigma^2_i \alpha^2_i} \bigg].
\end{equation}

Instead of the gaussian example, let us consider a Poisson distribution. The single event probability of some noise $\boldsymbol{\mathscr{N}}_i$ follows 
\begin{equation}
\mathbb{P}(\boldsymbol{\mathscr{N}}_i)
\end{equation}